# 推导和生成


列表推导还能更加通用，通用格式为
```python
[expression for target in iterable]
```

In [5]:
[(x + y) for x in range(5) if x % 2 == 0 for y in range(5) if y % 2 == 1]

[1, 3, 3, 5, 5, 7]

In [7]:
# 等效的for循环
res = []
for x in range(5):
    if x % 2 == 0:
        for y in range(5):
            if y % 2 == 1:
                res.append(x + y)
res

[1, 3, 3, 5, 5, 7]

In [28]:
list(map(lambda x: map(lambda y: x + y, filter(lambda y: y % 2 == 1, range(5))), filter((lambda x: x % 2 == 0), range(5))) # TODO 怎么实现呐

SyntaxError: unexpected EOF while parsing (<ipython-input-28-05bed1e04833>, line 1)

In [36]:
# map(lambda y: 1 + y, filter(lambda y:y%2==1, range(5)))
list(map(lambda x: list(map(lambda y: x + y, filter(lambda y:y%2==1, range(5)))), filter(lambda x: x % 2 == 0, range(5))))

[[1, 3], [3, 5], [5, 7]]

列表推导和矩阵

In [9]:
M = [[1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]]

In [10]:
[col+10 for row in M for col in row]

[11, 12, 13, 14, 15, 16, 17, 18, 19]

In [11]:
[[col + 10 for col in row] for row in M]

[[11, 12, 13], [14, 15, 16], [17, 18, 19]]

通常map调用比等效的for循环快两倍，而列表推导往往比map调用还要稍快一些

## 生成器函数和表达式

### 生成器函数: yield vs return

使用常规的def语句进行编写，但是使用yield语句一次返回一个结果，在每次结果产生之间挂起或恢复它们的状态

#### 状态挂起

和返回一个值并推出的常规函数不同，生成器函数能够自动挂起并生成值得时刻恢复之前得状态并继续函数的执行。  
生成器函数产生yield一个值，而不是返回return一个值，yield语句会挂起该函数并向调用者传回一个值，但同时也保留了足够的状态使函数能从它离开的地方继续。

In [12]:
def gensquares(N):
    for i in range(N):
        yield i**2

函数在每次循环时都会yield一个值，之后将其返还给调用者。当它被暂停后，它的上一个状态保存下来，包括变量i和N，并且在yield语句之后被马上收走控制权。

In [13]:
for i in gensquares(5):
    print(i, end=" ")

0 1 4 9 16

In [14]:
x = gensquares(5) # x是一个生成器对象，它有一个__next__方法，在获得一系列值得最后一个时，会引发StopIteration异常。

In [15]:
next(x)

0

In [16]:
next(x)

1

生成器对于大型程序而言，在内存使用和性能方向都更好。它们允许函数避免预先做好所有得工作，生成器将产生一系列值的时间分散到每一次的循环迭代中去。

#### send

send方法生成一系列结果的下一个元素，提供了一种调用者与生成器之间进行通信的方式，从而能够影响生成器的操作。

In [41]:
#生成器的send用法 generator.send(value)
def test():
    i = 1
    while i < 5:
        temp = yield i**2
        print(temp)
        i += 1

t = test()
#第一次运行只能使用next或者send(None)
print(t.__next__())
#send的作用相当于使生成器继续运行，并且传递的参数为yield的返回值(程序中即temp的值)
print(t.send("Hello World"))
print(t.__next__())#相当于send(None) 此时temp = None

1
Hello World
4
None
9


In [38]:
x = gener(3)

In [39]:
next(x)

0

In [40]:
print(x.send(4))

1


## 生成器表达式

生成器表达式就像一般的列表推到一样，支持列表推导的语法。生成器表达式不是在内存中构建结果，而是返回一个生成器对象，一个自动被创建的可迭代对象。支持可迭代协议，并在任意迭代语境中产生一个结果列表。

In [2]:
G = (x ** 2 for x in range(4))

In [3]:
iter(G) is G

True

In [4]:
next(G)

0

In [5]:
next(G)

1

生成器表达式是一种对内存空间的优化，不一次产生所有结果。 

## 生成器是单遍迭代对象

生成器函数和生成器表达式自身都是迭代器，支支持一次活跃迭代。当你尝试手动的使用多个迭代器来迭代结果流，它们将返回相同的位置

In [7]:
G = (x for x in range(5))

In [8]:
I1 = iter(G)

In [9]:
next(I1)

0

In [10]:
next(I1)

1

In [12]:
I2 = iter(G) # 其实和I1是同一个迭代器
next(I2)

3

而内置类型的生成器可以支持多个迭代器与多次迭代

In [13]:
L = [1, 2, 3, 4]

In [16]:
I1 = iter(L)
next(I1)
next(I1)

2

In [21]:
I2 = iter(L) # I2是新的迭代器
next(I2)

1

In [24]:
# python3的新方法
def gen(N):
    yield from range(N)
    # 等价于
    # for x in range(N): yield x

G = gen(5)
next(G)

0

## 内置类型，工具和类中的值生成

### 字典的键

In [18]:
D = {'a':1, 'b':2, 'c':3}
x = iter(D)

In [19]:
next(x)

'a'

In [20]:
for i in D:
    print(i, end=' ')

a b c